<a href="https://colab.research.google.com/github/mctavares/AI/blob/GenAISQLAgent/Copy_of_GenAISQLAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Gen AI SQL Agent
Ih this notebook we create a simple SQL Agent using Hugging Face and LangChain.


# References
## Data source:
* https://www.kaggle.com/code/dimarudov/data-analysis-using-sql

## SQLAlchemy:
* https://www.datacamp.com/tutorial/sqlalchemy-tutorial-examples
* https://www.slingacademy.com/article/sqlalchemy-how-to-bulk-insert-data-into-a-table/
* https://docs.sqlalchemy.org/en/20/orm/join_conditions.html
* https://www.sqlitetutorial.net/sqlite-go/create-table/

## Hugging Face
* https://huggingface.co/learn/cookbook/en/agent_text_to_sql




In [ ]:
# Check to see if dependencies are already installed
import transformers
import torch
import sqlalchemy
import transformers.agents

print('Hugging Face Transformers version:' + transformers.__version__)
print('PyTorch version:' + transformers.__version__)
print('sqlalchemy version:' + sqlalchemy.__version__)

In [ ]:
# Add here list of depedencies thta need to be installed
# !pip install sqlalchemy
!pip install fsspec
!pip install --upgrade transformers
!pip install transformers[agents]
!pip install --upgrade sqlalchemy

In [ ]:
# Let's create a simple database
import sqlalchemy as db

# Based on https://www.kaggle.com/code/alaasedeeq/european-soccer-database-with-sqlite3
# engine = db.create_engine("sqlite:///european_database.sqlite") # Uncomment if want db in disk

metadata = db.MetaData()

# Country table
country = db.Table('country', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False)
              )
# League table
league = db.Table('league', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Country_Id', db.Integer(),db.ForeignKey('country.Id')),
              db.Column('Name', db.String(255), nullable=False)
              )
engine = db.create_engine('sqlite:///:memory:') #, echo=True) # Comment out for verbose creation
metadata.create_all(engine)

# Connection will be open until we execute the agent since db is inn memory
conn = engine.connect()

In [ ]:
insert_stmt = country.insert().values([
{'Id':1, 'Name':'Belgium'},
{'Id':1729, 'Name':'England'},
{'Id':4769, 'Name':'France'},
{'Id':7809, 'Name':'Germany'},
{'Id':10257, 'Name':'Italy'},
{'Id':13274, 'Name':'Netherlands'},
{'Id':15722, 'Name':'Poland'},
{'Id':17642, 'Name':'Portugal'},
{'Id':19694, 'Name':'Scotland'},
{'Id':21518, 'Name':'Spain'},
{'Id':24558, 'Name':'Switzerland'}
])

conn.execute(insert_stmt)
output = conn.execute(country.select()).fetchall()

In [ ]:
insert_stmt = league.insert().values([
{'Id':1,'Country_Id':1,'Name':'Belgium Jupiler League'},
{'Id':1729,'Country_Id':1729,'Name':'England Premier League'},
{'Id':4769,'Country_Id':4769,'Name':'France Ligue 1'},
{'Id':7809,'Country_Id':7809,'Name':'Germany 1. Bundesliga'},
{'Id':10257,'Country_Id':10257,'Name':'Italy Serie A'},
{'Id':13274,'Country_Id':13274,'Name':'Netherlands Eredivisie'},
{'Id':15722,'Country_Id':15722,'Name':'Poland Ekstraklasa'},
{'Id':17642,'Country_Id':17642,'Name':'Portugal Liga ZON Sagres'},
{'Id':19694,'Country_Id':19694,'Name':'Scotland Premier League'},
{'Id':21518,'Country_Id':21518,'Name':'Spain LIGA BBVA	21518'},
{'Id':24558,'Country_Id':24558,'Name':'Switzerland Super League'}
])

conn.execute(insert_stmt)
output = conn.execute(league.select()).fetchall()

In [ ]:
# Hugging Face code
from transformers import ReactCodeAgent, HfApiEngine, Tool, tool

@tool
def query_tool(query: str) -> str:
  """
  Allows you to perform SQL queries on the table. Returns a string representation of the result.
  The table is named 'country'. Its description is as follows:
  Columns:
  - Id: INTEGER
  - Name: VARCHAR(16)

  Args:
  query: The query to perform. This should be correct SQL.
  """
  output = ""
  rows = conn.execute(db.text(query))
  for row in rows:
    output += "\n" + str(row)
  return output


agent = ReactCodeAgent(
    tools=[query_tool],
    llm_engine=HfApiEngine("meta-llama/Meta-Llama-3-8B-Instruct")
)

agent.run("Can you give me the name of a country that starts with the letter p?")

conn.close()
print ('Done')